In [41]:
import numpy as np
import random as rd

In [42]:
class Genes: # A chromosome is a set of genes, which are the courses, exam halls and time slots
    def __init__(self, courses, examHalls, timeSlots):
        self.courses = courses
        self.examHalls = examHalls
        self.timeSlots = timeSlots
    def getGenes(self):
        return self.courses, self.examHalls, self.timeSlots
    def printGene(self):
        print("( " + str(self.courses) + ", " + str(self.examHalls) + ", " + str(self.timeSlots) + " )")

In [43]:
def isConductionPossible(chromosome,examHall,timeSlot): # Check if the exam hall is available at the given time slot
    for gene in chromosome:
        if gene.examHalls == examHall and gene.timeSlots == timeSlot:
            return False
    return True

In [44]:
def initialization(popSize,courses,examHalls,timeSlots):
    population = []
    
    tempCourses = courses
    for i in range(popSize):
        chromosome = []
        for j in range(len(courses)):
            randCourse = np.random.choice(courses)
            
            index_to_remove = np.where(courses == randCourse)[0][0]
            courses = np.delete(courses, index_to_remove)
           
            randExamHall = np.random.choice(examHalls)
            randTimeSlot = np.random.choice(timeSlots)
            
            while not isConductionPossible(chromosome,randExamHall,randTimeSlot):
                prob = np.random.uniform(0,1)
                if prob < 0.5:
                    randExamHall = np.random.choice(examHalls)
                else:
                    randTimeSlot = np.random.choice(timeSlots)
            
            tempGene = Genes(randCourse,randExamHall,randTimeSlot)
            
            chromosome.append(tempGene)
        
        population.append(chromosome)
        courses = tempCourses
        
    return population

In [101]:
def printPopulation(pop):
    for chromosome in pop:
        chromosome = sortChromosomeonBasisofCourses(chromosome)
        print("Fitness Value: " + str(fitnessFunction(chromosome)))
        for gene in chromosome:
            gene.printGene()
        print("")


In [46]:
def isClash(C1,C2):
    global clashes
    for clash in clashes:
        if (C1 == clash[0] and C2 == clash[1]) or (C1 == clash[1] and C2 == clash[0]):
            return int(clash[2])
    return 0

In [47]:
def getExamsAtSameTimeSlots(chromosome):
    examsAtSameTimeSlots = []
    for i in range(len(chromosome)):
        for j in range(i+1,len(chromosome)):
            if chromosome[i].timeSlots == chromosome[j].timeSlots:
                examsAtSameTimeSlots.append([chromosome[i].courses,chromosome[j].courses])
    return examsAtSameTimeSlots

In [104]:
def isChromosomeCorrupted(chromosome):
    for i in range(len(chromosome)):
        for j in range(i+1,len(chromosome)):
            if chromosome[i].timeSlots == chromosome[j].timeSlots and chromosome[i].examHalls == chromosome[j].examHalls:
                return True
    return False

In [105]:
def fitnessFunction(chromosome):
    global courses, examHalls, timeSlots,clashes
    #To be implemented
    fitness = 0
    if isChromosomeCorrupted(chromosome):
        fitness+=1000 # If the chromosome is corrupted, then the fitness value is set to 1000. Corrupted chromosome means that after mutation or crossover, the chromosome is not feasible i.e the 2 different exams are conducted at the same time slot and same exam hall
    examsAtSameTimeSlots = getExamsAtSameTimeSlots(chromosome)
    for i in range(len(examsAtSameTimeSlots)):
        penalty=(isClash(examsAtSameTimeSlots[i][0],examsAtSameTimeSlots[i][1])//2)*100
        fitness += penalty
    return fitness
    

In [122]:
def tournamentSelection(size,population):
    newPopulation = []
    chosen = [] # To keep track of the chromosomes that have been selected so that they are not selected again
    
    for i in range(len(population)//size) :
        teams = [] # A team is a group of (size) number of chromosomes from which the best one will be selected
        for j in range (size):
            randomChoise = rd.randint(0,len(population)-1)
            
            while randomChoise in chosen:
                randomChoise = rd.randint(0,len(population)-1)
        
            chosen.append(randomChoise)
            teams.append(population[randomChoise])
            
        teams.sort(key=fitnessFunction)
        newPopulation.append(teams[0])
    
    return newPopulation
        
        
        

In [50]:
def sortChromosomeonBasisofCourses(chromosome):
    chromosome.sort(key=lambda x: x.courses)
    return chromosome

In [97]:
def crossOver(population):
    newPopulation = []
    for i in range(len(population)):
        parent1 = population[i]
        parent2 = population[(i+1)%len(population)]
        
        if np.random.random() < 0.8:
            crossOverPoint = np.random.randint(0,len(parent1))
            
            parent1 = sortChromosomeonBasisofCourses(parent1)
            parent2 = sortChromosomeonBasisofCourses(parent2)
            
            child1 = []
            child2 = []
            
            child1 = parent1[:crossOverPoint] + parent2[crossOverPoint:]
            child2 = parent2[:crossOverPoint] + parent1[crossOverPoint:]
            
            newPopulation.append(child1)
            newPopulation.append(child2)
        
        if np.random.random() < 0.2:
            newPopulation.append(parent1)
        if np.random.random() < 0.2:
            newPopulation.append(parent2)
    return newPopulation

In [52]:
def mutation(population,mutationRate):
    global courses, examHalls, timeSlots
    for chromosome in population:
        if np.random.random() < mutationRate:
            randIndex = np.random.randint(0,len(chromosome))
            chromosome[randIndex].timeSlots = np.random.choice(timeSlots)
            chromosome[randIndex].examHalls = np.random.choice(examHalls)
    return population
    

In [53]:
# Data for the problem
# --------------------
courses = np.array(["C1", "C2", "C3", "C4", "C5"]) 
examHalls = np.array(["H1", "H2"]) # Each exam hall can be used for a maximum of 6 hours
timeSlots = np.array(["T1", "T2", "T3"]) 
clashes = np.array([("C1", "C2",10),("C1", "C4",5),("C2", "C5",7),("C3", "C4",12),("C4", "C5",8)]) # (course1, course2, CommonStudents)
maxTimeSlotPerExamHall = 6 # Each exam hall can be used for a maximum of 6 hours
perExamTimeSlot = 3 # Each time slot is 2 hour long

In [136]:
# For 1 Generation 

population = initialization(100,courses,examHalls,timeSlots) # Step 1
population = tournamentSelection(5,population) # Step 2
population = crossOver(population) # Step 3
population = mutation(population,0.1) # Step 4

population.sort(key=fitnessFunction)
# printPopulation(population)

In [135]:
nGenerations = 100
population = initialization(1000,courses,examHalls,timeSlots) # Step 1
for i in range(nGenerations):
    population = tournamentSelection(2,population) # Step 2
    population = crossOver(population) # Step 3
    population = mutation(population,0.1) # Step 4
    
    # print("Size of Generation:",i+1,len(population))
population.sort(key=fitnessFunction)
printPopulation(population)

Fitness Value: 0
( C1, H1, T3 )
( C2, H1, T2 )
( C3, H2, T3 )
( C4, H2, T2 )
( C5, H2, T1 )

Fitness Value: 0
( C1, H1, T3 )
( C2, H1, T2 )
( C3, H2, T3 )
( C4, H2, T2 )
( C5, H2, T1 )

Fitness Value: 0
( C1, H1, T3 )
( C2, H1, T2 )
( C3, H2, T3 )
( C4, H2, T2 )
( C5, H2, T1 )

Fitness Value: 0
( C1, H1, T3 )
( C2, H1, T2 )
( C3, H2, T3 )
( C4, H2, T2 )
( C5, H2, T1 )

Fitness Value: 0
( C1, H1, T2 )
( C2, H1, T1 )
( C3, H2, T3 )
( C4, H2, T1 )
( C5, H2, T2 )

Fitness Value: 0
( C1, H2, T2 )
( C2, H2, T1 )
( C3, H1, T3 )
( C4, H1, T1 )
( C5, H2, T3 )

Fitness Value: 0
( C1, H2, T2 )
( C2, H2, T1 )
( C3, H1, T3 )
( C4, H1, T1 )
( C5, H2, T3 )

Fitness Value: 0
( C1, H2, T2 )
( C2, H2, T1 )
( C3, H1, T3 )
( C4, H1, T1 )
( C5, H2, T3 )

Fitness Value: 0
( C1, H1, T1 )
( C2, H1, T2 )
( C3, H1, T3 )
( C4, H2, T2 )
( C5, H2, T1 )

Fitness Value: 0
( C1, H1, T1 )
( C2, H1, T2 )
( C3, H1, T3 )
( C4, H2, T2 )
( C5, H2, T3 )

Fitness Value: 0
( C1, H2, T1 )
( C2, H2, T2 )
( C3, H1, T1 )
( C4, H1

If the entire population in a genetic algorithm becomes the same after 100 generations, it is possible that the algorithm has converged to a local optimum or has reached a plateau where the fitness of the population is not improving significantly. This phenomenon is called premature convergence.

To address premature convergence, some possible solutions include:

1. Adjusting selection pressure: Modifying the selection process to allow for more diversity in the population.

2. Improving crossover and mutation: Experimenting with different crossover and mutation operators or adjusting the parameters to allow for more genetic diversity.

3. Increasing population size: Increasing the number of individuals in the population can help maintain genetic diversity over multiple generations.

4. Modifying fitness function: Adjusting the fitness function to be more sensitive to small changes in the solution space can help maintain variation in the fitness values of the population.

### Tried the above methods but the Increasing the **population Size** worked for me.





Another problem that I faced was **tournament selection**. The problem is that let's say we have an initial population of size 100 and tournament size of 5 the new population would be of size 20 after tournament selection.
Then we go for crossOver and mutation. Suppose population increases a little and now our population is 30. For next generation if tournament size is 5 again, the population would be reduced to 6 only. 
If we do this for 100 generations the population would be be 0 after some iterations.
This is why I'm keeping the tournament size to 2, so that population survives after 100 iterations